In [1]:
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt 
%matplotlib inline

import examples_lib

In [2]:
a, b, x = examples_lib.get_quadratic_params(3)
examples_lib.get_quadratic_obj(x, a, b)

<tf.Tensor: shape=(), dtype=float64, numpy=0.9091416169164501>

In [3]:
mu, sigma, x, w = examples_lib.get_normal_params(10)
examples_lib.get_norm_ll_infl(x, mu, sigma, w)

<tf.Tensor: shape=(), dtype=float32, numpy=-10.754641>

In [4]:
x, mu_vec, sigma_vec, pi_vec, z_true = examples_lib.get_norm_cluster_params(3, 10)
e_z = z_true * 0.8 + 0.1
examples_lib.get_norm_clustering_mstep(x, mu_vec, sigma_vec, pi_vec, e_z)

<tf.Tensor: shape=(), dtype=float64, numpy=-1347.8231145018508>

In [5]:
theta, x_mat, y = examples_lib.get_logistic_params(10, 3)
examples_lib.get_logistic_ll(y, x_mat, theta)

<tf.Tensor: shape=(), dtype=float64, numpy=0.551107393829309>